In [1]:
import json
import requests
from bs4 import BeautifulSoup
import urllib
import codecs
from html.parser import HTMLParser
from collections import Counter
import dill
from requests_futures.sessions import FuturesSession
import re
import pandas as pd
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction import DictVectorizer
from sklearn.decomposition import PCA, SparsePCA
from sklearn.preprocessing import StandardScaler
from bokeh.io import output_notebook, show
from bokeh.layouts import column, row
from bokeh.models import Select
from bokeh.plotting import figure, curdoc, ColumnDataSource
from bokeh.palettes import Spectral10
import matplotlib.pyplot as plt
import mpld3
from sklearn.decomposition import NMF
import sklearn.metrics
import json
import altair as alt

In [ ]:
url = 'https://www.paulaschoice.com/ingredient-dictionary'
response = requests.get(url)
#response.text

In [ ]:
soup = BeautifulSoup(response.text)
categories = soup.select("select.category-filter option")[1:]
cat_links = [option['value'] for option in categories]
#cat_links

In [ ]:
base_url = "https://www.paulaschoice.com"
full_urls = [base_url + cat_link for cat_link in cat_links]
#full_urls

In [ ]:
session = FuturesSession(max_workers=8)
category_dict = {}
for url in full_urls:
    print(url)
    category = url.split("/")[-1]
    response = session.get(url)
    soup = BeautifulSoup(response.result().text)
    ingredients = soup.select("td.col-ingredient.ingredient-info h2 a")
    ingredients_names = [ingredient.text.lower() for ingredient in ingredients]
    category_dict[category] = ingredients_names

In [ ]:
dill.dump(category_dict, open("category_dict.pkd", 'wb'))
category_dict = dill.load(open("category_dict.pkd", 'rb'))

In [ ]:
url = "https://www.skincarisma.com/products/la-prairie/white-caviar-illuminating-cream/ingredient_list#info-section"
response = requests.get(url)
soup = BeautifulSoup(response.text)
ingredient_names = soup.select("div.col-12 table tbody tr td.align-middle small")

In [ ]:
def get_ingredients(url):
    response = session.get(url)
    soup = BeautifulSoup(response.result().text)
    ingredient_names = soup.select("div.col-12 table tbody tr td.align-middle small")
    ingredient_list = []
    ingredient_category = []
    for small in ingredient_names:
        tb = small.parent.text.strip().split("\n")
        two_rows = [text.strip() for text in tb if text.strip()]
        ingredient_name = two_rows[0]
        ingredient_list.append(ingredient_name)
        if len(two_rows) == 2:
            function_list = re.sub("\(|\)", "", two_rows[1]).split(",")
            function_list = [f for f in function_list if f]
            ingredient_category.extend(function_list)
    return ingredient_list, ingredient_category  

In [ ]:
beauty_products = pd.read_csv("beauty_products.csv")
beauty_products[:5]

In [ ]:
#get the data
session = FuturesSession(max_workers=8)
product_ingredients = {}
all_functions = []
product_funcions = {}
for _ , row in beauty_products.iterrows():
    name = row['Name']
    print(name)
    ingredients, category = get_ingredients(row['link'])
    product_ingredients[name] = ingredients
    all_functions.extend(category)
    function_count = Counter(category)
    product_funcions[name] = function_count

In [ ]:
dill.dump(product_funcions, open("product_func_dict.pkd", 'wb'))
dill.dump(product_ingredients, open("product_ingre_dict.pkd", 'wb'))
dill.dump(all_functions, open("all_func_dict.pkd", 'wb'))

In [2]:
product_funcions = dill.load(open("product_func_dict.pkd", 'rb'))
product_ingredients = dill.load(open("product_ingre_dict.pkd", 'rb'))
all_functions = dill.load(open("all_func_dict.pkd", 'rb'))

In [ ]:
all_ingredients = []
for key, val in product_ingredients.items():
    all_ingredients.extend(val)
ingredient_counts = Counter(all_ingredients)

In [ ]:
#plot top ingredients
ordered_counts = [(k,v) for k, v in sorted(ingredient_counts.items(), key=lambda x: x[1], reverse = True)]
df_top_ingredients = pd.DataFrame(ordered_counts[:10])
chart = sns.barplot(x=1, y=0, data=df_top_ingredients, orient="h")
#chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
chart.set_xlabel("")
chart.set_ylabel("")
chart.set_title("Top 10 ingredients", size=16)


In [ ]:
df = pd.DataFrame([product_ingredients]).transpose()
df.reset_index(inplace=True)

In [ ]:
merged_tb = pd.merge(beauty_products, df, left_on="Name", right_on="index")
merged_tb.to_csv("merged_data_table.csv",index=False)

In [10]:
merged_tb = pd.read_csv("merged_data_table.csv")
merged_tb['0'][1]

0      ['Niacin', 'Beta-Carotene', 'Decyl Oleate', 'P...
1      ['Glyceryl Polymethacrylate', 'Niacin', 'Disod...
2      ['C13-14 Isoparaffin', 'Hibiscus Abelmoschus E...
3      ['C13-14 Isoparaffin', 'Sesamum Indicum (Sesam...
4      ['Lanolin', 'Rosmarinus Officinalis (Rosemary)...
                             ...                        
145    ['Glyceryl Stearate', 'Zizyphus Jujuba Fruit E...
146    ['Scutellaria Galericulata Extract', 'Glycyrrh...
147    ['C12-15 Alkyl Benzoate', 'Glyceryl Stearate',...
148    ['1,2-Hexanediol', 'C12-20 Alkyl Glucoside', '...
149    ['Galactoarabinan', 'Glycyrrhiza Glabra (Licor...
Name: 0, Length: 150, dtype: object

In [33]:

a = merged_tb['0'][1]
b = merged_tb['0'][2]
ingre1 = [x.strip("'") for x in a.strip("][").split(", ")]
ingre2 = [x.strip("'") for x in b.strip("][").split(", ")]
c = set(ingre1).intersection(ingre2)
c1 = set(ingre1) - set(ingre2)

In [34]:
c

{'Butylene Glycol',
 'Calcium Gluconate',
 'Citronellol',
 'Citrus Aurantifolia (Lime) Peel Extract',
 'Copper Gluconate',
 'Cyclopentasiloxane',
 'Dimethicone',
 'Disodium EDTA',
 'Eucalyptol',
 'Fragrance',
 'Fucus Vesiculosus Extract',
 'Geraniol',
 'Helianthus Annuus (Sunflower) Seedcake',
 'Hydroxycitronellal',
 'Limonene',
 'Linalool',
 'Magnesium Gluconate',
 'Medicago Sativa (Alfalfa) Seed Powder',
 'Niacin',
 'Phenoxyethanol',
 'Potassium Gluconate',
 'Potassium Sorbate',
 'Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal',
 'Sesamum Indicum (Sesame) Seed',
 'Sodium Gluconate',
 'Sodium Hyaluronate',
 'Tocopheryl Acetate (Vitamin E)',
 'Tocopheryl Succinate',
 'Zinc Gluconate'}

In [37]:
c1

{'Aloe Barbadensis Leaf Water',
 'Ammonium Acryloyldimethyltaurate/VP Copolymer',
 'Caffeine',
 'Glycerin',
 'Glyceryl Polymethacrylate',
 'Hydrogenated Lecithin',
 'Oleth-10',
 'PEG-8',
 'Palmitoyl Oligopeptide',
 'Sesamum Indicum (Sesame) Seed Oil',
 'Sorbitol',
 'Sucrose',
 'Trehalose'}

In [38]:
ingre1

['Glyceryl Polymethacrylate',
 'Niacin',
 'Disodium EDTA',
 'Magnesium Gluconate',
 'Fucus Vesiculosus Extract',
 'Sodium Gluconate',
 'Sorbitol',
 'Sucrose',
 'Prunus Amygdalus Dulcis (Sweet Almond) Seed Meal',
 'Ammonium Acryloyldimethyltaurate/VP Copolymer',
 'Eucalyptol',
 'Medicago Sativa (Alfalfa) Seed Powder',
 'Sesamum Indicum (Sesame) Seed',
 'Sesamum Indicum (Sesame) Seed Oil',
 'Caffeine',
 'Calcium Gluconate',
 'Butylene Glycol',
 'Sodium Hyaluronate',
 'Glycerin',
 'Trehalose',
 'Palmitoyl Oligopeptide',
 'Potassium Gluconate',
 'Helianthus Annuus (Sunflower) Seedcake',
 'Oleth-10',
 'Copper Gluconate',
 'Hydrogenated Lecithin',
 'Dimethicone',
 'Citrus Aurantifolia (Lime) Peel Extract',
 'Cyclopentasiloxane',
 'Zinc Gluconate',
 'Tocopheryl Acetate (Vitamin E)',
 'Potassium Sorbate',
 'PEG-8',
 'Phenoxyethanol',
 'Linalool',
 'Citronellol',
 'Limonene',
 'Geraniol',
 'Hydroxycitronellal',
 'Fragrance',
 'Aloe Barbadensis Leaf Water',
 'Tocopheryl Succinate']

In [ ]:
merged_json = merged_tb.to_dict('records')
merged_json

In [9]:
with open("data.json", "w") as f:
    json.dump(merged_json, f)

In [12]:
ingredient_ls = []
product_name = []
for k, v in product_ingredients.items():
    product_name.append(k)
    ingredient_dict = {}
    for each_ingre in v:
        ingredient_dict[each_ingre] = 1
    ingredient_ls.append(ingredient_dict)

In [13]:
transform_ingre = DictVectorizer()
ingredient_matrix = transform_ingre.fit_transform(ingredient_ls).toarray()
col_names = transform_ingre.get_feature_names()
df_ingredient = pd.DataFrame(ingredient_matrix, columns=col_names, index = product_name)
df_ingredient.to_csv("ingredient_matrix.csv")

In [4]:
df_ingredient = pd.read_csv("ingredient_matrix.csv", index_col=0)

In [85]:
names = [i for i in df_ingredient.index]
n = names[:6]

In [78]:
#df_ingredient.reset_index(inplace=True)
new=df_ingredient.query(f'index in {n}')
new.set_index("index")

,"1,2-Hexanediol",2-O-Ethyl Ascorbic Acid,3-Aminopropane Sulfonic Acid,Acacia Decurrens Flower Wax,Acacia Senegal Gum,Acanthopanax Senticosus (Eleuthero) Root Extract,Acer Saccharum (Sugar Maple) Extract,Acetic Acid,Acetyl Carnitine HCl,Acetyl Cysteine,...,Zingiber Aromaticus Extract,Zingiber Officinale (Ginger) Root Extract,Zizyphus Jujuba Fruit Extract,imperata Cylindrica Root Extract,sh-Oligopeptide-1,sh-Oligopeptide-2,sh-Polypeptide-1,sh-Polypeptide-11,sh-Polypeptide-9,t-Butyl Alcohol
index,,,,,,,,,,,,,,,,,,,,,
La Mer Crème De La Mer,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
La Mer The Moisturizing Gel Cream,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
La Mer The Moisturizing Lotion,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
La Mer The Moisturizing Soft Cream,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Weleda Skin Food,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Nivea Crème,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [5]:
ingredient_pca = PCA(n_components=2)
PCs = ingredient_pca.fit_transform(df_ingredient)
df_PCs = pd.DataFrame(PCs, columns=['PC1', 'PC2'])
#plt.scatter(df_PCs['PC1'], df_PCs['PC2'])
#plt.title("PCA on ingredients")
#plt.xlabel("PC1")
#plt.ylabel("PC2")

In [10]:
df_PCs['Name'] = df_ingredient.index
df_PCs['Brand'] = merged_tb['Brand']
df_PCs['Function'] = merged_tb['Function']
df_PCs['Category'] = merged_tb['Category']
df_PCs

,PC1,PC2,Name,Brand,Function,Category
0,1.136743,2.521174,La Mer Crème De La Mer,La Mer,moisturizer,cream
1,2.254745,2.837838,La Mer The Moisturizing Gel Cream,La Mer,moisturizer,cream
2,2.914197,2.066344,La Mer The Moisturizing Lotion,La Mer,moisturizer,cream
3,3.760883,3.212493,La Mer The Moisturizing Soft Cream,La Mer,moisturizer,cream
4,-0.664725,0.147844,Weleda Skin Food,Weleda,moisturizer,cream
...,...,...,...,...,...,...
145,-0.802653,0.006273,Tatcha Violet-C Radiance Mask,Tatcha,brightening,facial mask (cream)
146,-1.451480,0.529775,Mario Badescu Whitening Mask,Mario Badescu,brightening,facial mask (cream)
147,0.711755,-0.823603,GlamGlow Flashmud Brightening Treatment,GlamGlow,brightening,facial mask (cream)
148,-0.279550,0.223535,Belif Overnight brightening mask,Belif,brightening,facial mask (cream)


In [63]:
input_dropdown = alt.binding_select(options=['Brand','Function','Category'])
selection = alt.selection_single(name='Color By', fields=['column'], bind=input_dropdown, init={'column':'Function'})
alt.Chart(df_PCs).transform_fold(['Brand', 'Function', 'Category'], as_=['column', 'value']).transform_filter(selection).mark_point().encode(x="PC1:Q", y="PC2:Q", color="value:N", column="column:N", tooltip=['Name:N']).add_selection(selection)

alt.Chart(...)

In [62]:
import altair as alt
import pandas as pd
import numpy as np

df = pd.DataFrame({
    'x': np.random.randn(100),
    'y': np.random.randn(100),
    'c1': np.random.randint(0, 3, 100),
    'c2': np.random.randint(0, 3, 100),
    'c3': np.random.randint(0, 3, 100),
})

selector = alt.selection_single(
    name='Color by',
    fields=['column'],
    bind=alt.binding_select(options=['Brand','Function','Category']),
    init={'column': 'Function'}
)

alt.Chart(df_PCs).transform_fold(
    ['Brand','Function','Category'], as_=['column', 'value']
).transform_filter(
    selector
).mark_point().encode(
    x='PC1:Q',
    y='PC2:Q',
    color='value:N',
    column='column:N'
).add_selection(
    selector
)

alt.Chart(...)

In [ ]:
model = NMF(n_components=2, init='random', random_state=0)
W = model.fit_transform(df_ingredient[:50])
df_W = pd.DataFrame(W, columns=["PC1", "PC2"])
plt.scatter(df_W['PC1'], df_W['PC2'])

In [9]:
pairwise_matrix = pd.DataFrame(sklearn.metrics.pairwise_distances(df_ingredient.to_numpy(), metric="jaccard"), columns=df_ingredient.index, index=df_ingredient.index)
first_col = pairwise_matrix[pairwise_matrix.index[0]]
first_col[first_col.index==pairwise_matrix.index[3]].values

/Users/weiqingwang/opt/anaconda3/lib/python3.7/site-packages/sklearn/metrics/pairwise.py:1765: DataConversionWarning: Data was converted to boolean for metric jaccard
  warnings.warn(msg, DataConversionWarning)


array([0.7628866])

In [95]:
col = pairwise_matrix[n[3]]
sorted_col = col.sort_values()
list(sorted_col.items())

[('La Mer The Moisturizing Soft Cream', 0.0),
 ('La Mer The Moisturizing Lotion', 0.59375),
 ('La Mer The Lifting and Firming Mask', 0.6616541353383458),
 ('La Mer The Moisturizing Gel Cream', 0.6629213483146067),
 ('La mer The Regenerating Serum', 0.7),
 ('La Mer The Intensive Revitalizing Mask', 0.72),
 ('La mer The Concentrate', 0.7553191489361702),
 ('La Mer  The Eye Concentrate', 0.7559055118110236),
 ('La Mer The Eye Balm Intense', 0.7560975609756098),
 ('La Mer Crème De La Mer', 0.7628865979381443),
 ('La Prairie Skin Caviar Luxe Eye Lift Cream', 0.7984496124031008),
 ('La Mer The Hydrating Facial', 0.8053097345132744),
 ('La Prairie Cellular Hydralift Firming Mask ', 0.8222222222222222),
 ('La Prairie Anti-Aging Day Cream SPF 30', 0.8345864661654135),
 ('La Prairie Cellular Radiance Night Cream', 0.8391608391608392),
 ('La Prairie Skin Caviar Luxe Cream', 0.8489208633093526),
 ('La Prairie Anti-Aging Night Cream', 0.8492063492063492),
 ('Lancome Advanced Génifique Eye Cream', 0

In [ ]:
def concat(rows):
    category_ingredient=[]
    for row in rows:
        category_ingredient.extend(row)
    return category_ingredient

ingredient_by_category = pd.DataFrame(merged_tb.groupby(['Category'])[0].apply(concat))

In [ ]:
count_ingredients = pd.DataFrame(ingredient_by_category[0].apply(Counter))
count_ingredients.reset_index(inplace=True)

In [ ]:
for _ , row in count_ingredients.iterrows():
    counts = row[0]
    cat = row['Category']
    ordered_counts = [(k,v) for k, v in sorted(counts.items(), key=lambda x: x[1], reverse = True)]
    df_top_ingredients = pd.DataFrame(ordered_counts[:10])
    chart = sns.barplot(x=1, y=0, data=df_top_ingredients, orient="h")
    #chart.set_xticklabels(chart.get_xticklabels(), rotation=90)
    chart.set_xlabel("")
    chart.set_ylabel("")
    title = cat + " top 10 ingredients"
    chart.set_title(title, size=16)
    filename = "images/"+ cat + "_top10.png"
    fig = chart.get_figure()
    fig.savefig(filename, dpi=300, bbox_inches="tight")

In [ ]:
col_names = []
dict_list = []
for k, v in product_funcions.items():
    col_names.append(k)
    dict_list.append(v)



In [ ]:
transformer = DictVectorizer()

function_matrix = transformer.fit_transform(dict_list).toarray()
feature_names = transformer.get_feature_names()


In [ ]:
function_df = pd.DataFrame(function_matrix, index=col_names, columns=feature_names)

In [ ]:
X = StandardScaler().fit_transform(function_matrix)
pca = PCA(n_components=2)
principalComponents = pca.fit_transform(X)
df_pca = pd.DataFrame(principalComponents, columns=['PC1', 'PC2'], index = col_names)


In [ ]:
category_order = []
function_order = []
brand_order = []
for each_name in col_names:
    row = merged_tb[merged_tb['Name']==each_name]
    category_ = row['Category'].values
    category_order.extend(category_)
    function_ = row['Function'].values
    function_order.extend(function_)
    brand_ = row['Brand'].values
    brand_order.extend(brand_)